# 1. Set UP

In [ ]:
# we did some research around stacked ensembles and found some helpful and interesting links

https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

http://blog.keyrus.co.uk/ensembling_ml_models.html

https://www.dataquest.io/blog/introduction-to-ensembles/

https://www.kaggle.com/nicapotato/titanic-voting-pipeline-stack-and-guide

In [42]:
import os
import numpy as np
import pandas as pd
import pickle
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [9]:
from category_encoders import *

In [10]:
h2o.remove_all

<function h2o.h2o.remove_all(retained=None)>

In [11]:
# Set directories
print(os.getcwd())
dirRawData = "../input/"
dirPData   = "../PData/"
dirPOutput = "../POutput/"

/home/jovyan/Projects/Fraud/PCode


In [12]:
f_name = dirPData + '01_df_250k.pickle'

with (open(f_name, "rb")) as f:
    dict_ = pickle.load(f)

df_train = dict_['df_train']
df_test  = dict_['df_test']

del f_name, dict_

f_name = dirPData + '01_vars.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)

vars_ind_numeric     = dict_['vars_ind_numeric']
vars_ind_hccv        = dict_['vars_ind_hccv']
vars_ind_categorical = dict_['vars_ind_categorical']
vars_notToUse        = dict_['vars_notToUse']
var_dep              = dict_['var_dep']

del f_name, dict_

In [6]:
f_name = dirPData + '01_df_250k.pickle'

with (open(f_name, "rb")) as f:
    dict_ = pickle.load(f)

df_test  = dict_['df_test']

del f_name, dict_


In [ ]:
### Data cleaning same as part 3 

In [13]:
vars_ind_categorical.remove('c02')

In [14]:
vars_ind_numeric.remove('e16')

In [15]:
[var for var in vars_ind if var not in vars_ind_categorical]

NameError: name 'vars_ind' is not defined

In [16]:
vars_ind_categorical=[var for var in vars_ind_categorical if var not in vars_notToUse]

In [17]:
## constant variables
vars_notToUse.append('c02')
vars_notToUse.append('e16')

In [18]:
vars_ind_categorical

['a01',
 'a02',
 'a03',
 'a10',
 'a12',
 'a13',
 'a16',
 'a17',
 'a18',
 'a19',
 'a20',
 'b02',
 'b03',
 'b04',
 'b07',
 'c04',
 'c05',
 'c06',
 'c07',
 'c08',
 'c09',
 'e01',
 'e03',
 'e11',
 'e13',
 'e14',
 'e21',
 'e22',
 'e24',
 'e25',
 'e17',
 'e18',
 'e19',
 'e20',
 'f03',
 'f04',
 'f05',
 'f07',
 'f09',
 'f27',
 'f29',
 'f30',
 'f33',
 'f34',
 'f10']

In [110]:
df_test.shape

(296690, 96)

In [19]:
vars_ind = vars_ind_categorical + vars_ind_numeric

In [20]:
var_dep

['target']

In [21]:
#standardising before splines seems to mess it up 
for var in vars_ind_numeric:
    x = df_train[var]
    x -= np.mean(x, axis=0)
    x /= np.sqrt(np.mean(x ** 2, axis=0))
    df_train[var] = x

In [22]:
for var in vars_ind_numeric:
    x = df_test[var]
    x -= np.mean(x, axis=0)
    x /= np.sqrt(np.mean(x ** 2, axis=0))
    df_test[var] = x

In [115]:
df_train

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f21,f22,f23,f24,f25,f26,f28,f31,f32,unique_id
0,1,D,C,H,Z,C,A,Z,Z,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,1
1,1,D,C,H,Z,C,A,Z,Z,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,2
2,0,D,C,H,Z,C,A,Z,Z,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,3
3,1,B,H,H,N,A,A,Y,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,-2.003223,0.098285,4
4,0,B,H,H,N,A,A,Y,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1,B,A,H,N,A,A,Y,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,2.284880,287338
249996,0,B,A,B,N,A,A,N,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,287339
249997,1,B,A,B,N,A,A,N,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,287340
249998,0,B,A,B,N,A,A,N,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,287341


In [23]:
#spline

vars_ind_tospline = df_train[vars_ind_numeric].columns[(df_train[vars_ind_numeric].nunique() > 8)].tolist()

In [24]:
vars_ind_tospline

['a04',
 'a15',
 'b01',
 'b05',
 'c01',
 'c03',
 'd01',
 'd02',
 'd03',
 'e02',
 'e04',
 'e05',
 'e06',
 'e08',
 'e09',
 'e12',
 'e15',
 'f01',
 'f02',
 'f06',
 'f11',
 'f13']

In [25]:
df_train

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f21,f22,f23,f24,f25,f26,f28,f31,f32,unique_id
0,1,D,C,H,Z,C,A,Z,Z,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,1
1,1,D,C,H,Z,C,A,Z,Z,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,2
2,0,D,C,H,Z,C,A,Z,Z,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,3
3,1,B,H,H,N,A,A,Y,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,-2.003223,0.098285,4
4,0,B,H,H,N,A,A,Y,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1,B,A,H,N,A,A,Y,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,2.284880,287338
249996,0,B,A,B,N,A,A,N,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,287339
249997,1,B,A,B,N,A,A,N,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,287340
249998,0,B,A,B,N,A,A,N,N,F,...,-0.085789,-0.208867,-0.046292,-0.079624,-0.081068,-0.078524,0.181275,0.110374,0.098285,287341


In [26]:
def fn_tosplines(varname,x):
    ptiles = np.percentile(x, [10, 20, 40, 60, 80, 90])
    ptiles = np.unique(ptiles)
    df_ptiles = pd.DataFrame({var: x})
    for idx, ptile in enumerate(ptiles):
        df_ptiles[var + '_' + str(idx)] = np.maximum(0, x - ptiles[idx])
    return(df_ptiles)

In [27]:
for var in vars_ind_tospline:
    df_ptiles = fn_tosplines(var,df_train[var])
    df_train.drop(columns=[var], inplace=True)
    vars_ind_numeric.remove(var)
    df_train = pd.concat([df_train, df_ptiles], axis=1, sort=False)
    vars_ind_numeric.extend(df_ptiles.columns.tolist())

In [28]:
for var in vars_ind_tospline:
    df1_ptiles = fn_tosplines(var,df_test[var])
    df_test.drop(columns=[var], inplace=True)
    vars_ind_numeric.remove(var)
    df_test = pd.concat([df_test, df1_ptiles], axis=1, sort=False)
    vars_ind_numeric.extend(df1_ptiles.columns.tolist())

In [29]:
len(vars_ind_numeric)

270

In [123]:
#srs_missing = df_combined.isnull().sum(axis=0)
#srs_missing[srs_missing > 20] 

In [30]:
df_test

,a01,a02,a03,a10,a12,a13,a16,a17,a18,a19,...,f11_3,f11_4,f11_5,f13,f13_0,f13_1,f13_2,f13_3,f13_4,f13_5
0,B,A,B,N,A,A,Y,N,F,N,...,0.000000,0.000000,0.000000,0.784807,2.296967,1.531312,0.765656,0.382828,0.0,0.0
1,D,C,H,Z,C,A,Z,Z,F,N,...,0.973356,0.486678,0.243339,0.401979,1.914139,1.148484,0.382828,0.000000,0.0,0.0
2,D,C,H,Z,C,A,Z,Z,F,N,...,0.000000,0.000000,0.000000,0.401979,1.914139,1.148484,0.382828,0.000000,0.0,0.0
3,B,A,B,N,A,A,Y,N,F,N,...,0.486678,0.000000,0.000000,0.401979,1.914139,1.148484,0.382828,0.000000,0.0,0.0
4,B,A,B,N,A,A,Y,N,F,N,...,0.243339,0.000000,0.000000,0.019151,1.531312,0.765656,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296685,B,A,H,N,A,A,Y,N,F,N,...,0.730017,0.243339,0.000000,-0.746505,0.765656,0.000000,0.000000,0.000000,0.0,0.0
296686,B,A,H,N,A,A,Y,N,F,N,...,0.243339,0.000000,0.000000,0.401979,1.914139,1.148484,0.382828,0.000000,0.0,0.0
296687,B,A,H,N,A,A,Y,N,F,N,...,0.000000,0.000000,0.000000,0.401979,1.914139,1.148484,0.382828,0.000000,0.0,0.0
296688,B,A,C,N,A,A,Y,N,F,N,...,0.000000,0.000000,0.000000,0.401979,1.914139,1.148484,0.382828,0.000000,0.0,0.0


In [31]:
vars_ind = vars_ind_categorical + vars_ind_numeric

In [32]:
y_train = df_train[var_dep].values.ravel()

In [127]:
vars_ind_hccv

['e17', 'e18', 'e19', 'f10']

In [33]:
#binary encoder

enc = BinaryEncoder(cols=['e17','e18','e19','f10'])

enc.fit(df_train[vars_ind_hccv])
df_encoded = enc.transform(df_train[vars_ind_hccv])
df_encoded[vars_ind_hccv] = df_train[vars_ind_hccv]
df_encoded.head()

#df_encoded = enc.fit_transform(df_train, y_train)

/opt/conda/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,e17_0,e17_1,e17_2,e17_3,e17_4,e17_5,e17_6,e17_7,e18_0,e18_1,...,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11,e17,e18,e19,f10
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,38CF0,D556D,D2AFA,CHO
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,38CF0,D556D,D2AFA,AIR
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,38CF0,D556D,D2AFA,CHE
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,B6A15,E7317,CE2BB,CPH
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,B6A15,E7317,CE2BB,CUO


In [34]:
df_encoded

,e17_0,e17_1,e17_2,e17_3,e17_4,e17_5,e17_6,e17_7,e18_0,e18_1,...,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11,e17,e18,e19,f10
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,38CF0,D556D,D2AFA,CHO
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,38CF0,D556D,D2AFA,AIR
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,38CF0,D556D,D2AFA,CHE
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,B6A15,E7317,CE2BB,CPH
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,B6A15,E7317,CE2BB,CUO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0,0,0,0,0,1,0,1,0,0,...,0,1,0,1,1,0,C93B9,2557E,2EDD2,CNS
249996,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,1,C93B9,513C8,B4FF3,CUO
249997,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,1,C93B9,513C8,B4FF3,CUO
249998,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,1,C93B9,513C8,B4FF3,CUO


In [35]:
df_encoded=df_encoded.drop(['e17', 'e18','e19','f10'], axis=1)

In [36]:
vars_ind_categorical.extend(df_encoded.columns.tolist())

In [37]:
df_train = pd.concat([df_train, df_encoded], axis=1, sort=False)

In [38]:
df_train

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f10_2,f10_3,f10_4,f10_5,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11
0,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,0,0,0,0,0,0,1
1,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,0,0,0,0,0,1,0
2,0,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,0,0,0,0,0,1,1
3,1,B,H,H,N,A,A,Y,N,F,...,0,0,0,0,0,0,0,1,0,0
4,0,B,H,H,N,A,A,Y,N,F,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1,B,A,H,N,A,A,Y,N,F,...,0,1,0,1,0,1,0,1,1,0
249996,0,B,A,B,N,A,A,N,N,F,...,0,0,0,0,0,0,0,1,0,1
249997,1,B,A,B,N,A,A,N,N,F,...,0,0,0,0,0,0,0,1,0,1
249998,0,B,A,B,N,A,A,N,N,F,...,0,0,0,0,0,0,0,1,0,1


In [39]:
df_train=df_train.drop(['e17', 'e18','e19','f10'], axis=1)

In [40]:
#binary encoder for test data

enc = BinaryEncoder(cols=['e17','e18','e19','f10'])

enc.fit(df_test[vars_ind_hccv])
df_encoded = enc.transform(df_test[vars_ind_hccv])
df_encoded[vars_ind_hccv] = df_test[vars_ind_hccv]
df_encoded.head()


/opt/conda/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,e17_0,e17_1,e17_2,e17_3,e17_4,e17_5,e17_6,e17_7,e18_0,e18_1,...,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11,e17,e18,e19,f10
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,920BD,11250,ECD18,CKG
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,F33BC,5614E,NaN,AIR
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,1,F33BC,5614E,NaN,AEL
3,0,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,861C8,9076B,463D0,CUO
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,1,861C8,9076B,463D0,AWV


In [41]:
df_encoded=df_encoded.drop(['e17', 'e18','e19','f10'], axis=1)

In [43]:
df_test = pd.concat([df_test, df_encoded], axis=1, sort=False)

In [44]:
df_test=df_test.drop(['e17', 'e18','e19','f10'], axis=1)

In [45]:
df_test

,a01,a02,a03,a10,a12,a13,a16,a17,a18,a19,...,f10_2,f10_3,f10_4,f10_5,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11
0,B,A,B,N,A,A,Y,N,F,N,...,0,0,0,0,0,0,0,0,0,1
1,D,C,H,Z,C,A,Z,Z,F,N,...,0,0,0,0,0,0,0,0,1,0
2,D,C,H,Z,C,A,Z,Z,F,N,...,0,0,0,0,0,0,0,0,1,1
3,B,A,B,N,A,A,Y,N,F,N,...,0,0,0,0,0,0,0,1,0,0
4,B,A,B,N,A,A,Y,N,F,N,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296685,B,A,H,N,A,A,Y,N,F,N,...,0,0,0,0,1,1,0,0,0,0
296686,B,A,H,N,A,A,Y,N,F,N,...,0,1,1,0,1,0,1,1,0,0
296687,B,A,H,N,A,A,Y,N,F,N,...,0,0,0,0,0,1,0,0,0,0
296688,B,A,C,N,A,A,Y,N,F,N,...,0,0,0,0,0,0,0,0,1,1


In [46]:
vars_ind_numeric

['a05',
 'a06',
 'a07',
 'a08',
 'a09',
 'a11',
 'a14',
 'b06',
 'e07',
 'e23',
 'f08',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f25',
 'f26',
 'f28',
 'f31',
 'f32',
 'a04_0',
 'a04_1',
 'a04_2',
 'a04_3',
 'a15_0',
 'a15_1',
 'a15_2',
 'a15_3',
 'a15_4',
 'a15_5',
 'b01_0',
 'b01_1',
 'b01_2',
 'b01_3',
 'b01_4',
 'b01_5',
 'b05_0',
 'b05_1',
 'b05_2',
 'b05_3',
 'b05_4',
 'b05_5',
 'c01_0',
 'c01_1',
 'c01_2',
 'c01_3',
 'c03_0',
 'c03_1',
 'c03_2',
 'c03_3',
 'c03_4',
 'd01_0',
 'd01_1',
 'd01_2',
 'd01_3',
 'd01_4',
 'd02_0',
 'd02_1',
 'd02_2',
 'd02_3',
 'd03_0',
 'd03_1',
 'd03_2',
 'e02_0',
 'e02_1',
 'e02_2',
 'e02_3',
 'e02_4',
 'e02_5',
 'e04_0',
 'e04_1',
 'e04_2',
 'e04_3',
 'e04_4',
 'e04_5',
 'e05_0',
 'e05_1',
 'e05_2',
 'e05_3',
 'e05_4',
 'e05_5',
 'e06_0',
 'e06_1',
 'e06_2',
 'e06_3',
 'e08_0',
 'e08_1',
 'e08_2',
 'e08_3',
 'e08_4',
 'e09_0',
 'e09_1',
 'e09_2',
 'e09_3',
 'e09_4',
 'e12_0',
 'e12_1',
 'e12_2',
 'e12_3',
 '

In [ ]:
y_train

In [47]:
vars_ind_categorical.remove('e17')

vars_ind_categorical.remove('e18')
vars_ind_categorical.remove('e19')
vars_ind_categorical.remove('f10')


In [48]:
vars_ind_numeric.remove('e09_4') # this spline does not appear in test set

In [49]:
vars_ind_numeric.remove('f01_4')

In [73]:
vars_ind_numeric.remove('d02_2')

In [74]:
vars_ind_numeric.remove('d02_3')

In [50]:
vars_ind

['a01',
 'a02',
 'a03',
 'a10',
 'a12',
 'a13',
 'a16',
 'a17',
 'a18',
 'a19',
 'a20',
 'b02',
 'b03',
 'b04',
 'b07',
 'c04',
 'c05',
 'c06',
 'c07',
 'c08',
 'c09',
 'e01',
 'e03',
 'e11',
 'e13',
 'e14',
 'e21',
 'e22',
 'e24',
 'e25',
 'e17',
 'e18',
 'e19',
 'e20',
 'f03',
 'f04',
 'f05',
 'f07',
 'f09',
 'f27',
 'f29',
 'f30',
 'f33',
 'f34',
 'f10',
 'a05',
 'a06',
 'a07',
 'a08',
 'a09',
 'a11',
 'a14',
 'b06',
 'e07',
 'e23',
 'f08',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f25',
 'f26',
 'f28',
 'f31',
 'f32',
 'a04_0',
 'a04_1',
 'a04_2',
 'a04_3',
 'a15_0',
 'a15_1',
 'a15_2',
 'a15_3',
 'a15_4',
 'a15_5',
 'b01_0',
 'b01_1',
 'b01_2',
 'b01_3',
 'b01_4',
 'b01_5',
 'b05_0',
 'b05_1',
 'b05_2',
 'b05_3',
 'b05_4',
 'b05_5',
 'c01_0',
 'c01_1',
 'c01_2',
 'c01_3',
 'c03_0',
 'c03_1',
 'c03_2',
 'c03_3',
 'c03_4',
 'd01_0',
 'd01_1',
 'd01_2',
 'd01_3',
 'd01_4',
 'd02_0',
 'd02_1',
 'd02_2',
 'd02_3',
 'd03_0',
 'd03_1',
 'd03_2',
 '

In [51]:
vars_ind = vars_ind_categorical + vars_ind_numeric

In [52]:
# add features engineered through division of numerics for train set
df_train['x0'] = df_train['e04']/df_train['e06']
df_train['x1'] = df_train['f13']/df_train['f02']
df_train['x2'] = df_train['e09']/df_train['f11']

# manually add interactions as the interaction_pairs argument didn't work in h2o
df_train['x3'] = df_train['e04']*df_train['e06']
df_train['x4'] = df_train['f13']*df_train['f02']
df_train['x5'] = df_train['e09']*df_train['f11']

In [53]:
# same as above for test set
df_test['x0'] = df_test['e04']/df_test['e06']
df_test['x1'] = df_test['f13']/df_test['f02']
df_test['x2'] = df_test['e09']/df_test['f11']

# same as above for test set
df_test['x3'] = df_test['e04']*df_test['e06']
df_test['x4'] = df_test['f13']*df_test['f02']
df_test['x5'] = df_test['e09']*df_test['f11']

In [54]:
# add new vars to vars_ind index
vars_ind.append('x0')
vars_ind.append('x1')
vars_ind.append('x2')
vars_ind.append('x3')
vars_ind.append('x4')
vars_ind.append('x5')

In [55]:
#check length of variables list
len(vars_ind)

362

In [56]:
# make sure no variables are repeated so we use set and then pass to list again
myset=set(vars_ind)

print(len(myset))

vars_ind=list(myset)


253


In [57]:
var_dep

['target']

In [58]:
df_all=df_train.copy()# create smaller dataset for faster computation

In [61]:
df_all=df_all.sample(10000,random_state=2021)

In [62]:
df_test_50=df_test.sample(10000,random_state=2021)

In [63]:
df_all

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f10_8,f10_9,f10_10,f10_11,x0,x1,x2,x3,x4,x5
132715,0,B,H,H,N,A,A,Y,N,F,...,0,0,0,1,1.403348,1.806160,0.292300,1.577773,1.320659,0.229985
44369,1,B,A,B,N,A,A,Y,N,F,...,0,0,1,0,102.661026,-0.399090,0.318926,0.016831,-0.340992,0.554207
48214,1,B,A,B,N,A,A,Y,N,F,...,0,1,0,1,1.043337,0.264350,0.205276,1.173015,0.514797,0.140793
9636,1,B,A,B,N,A,A,Y,N,F,...,0,1,1,1,0.356044,-2.577813,-0.989194,0.400297,-0.925328,-0.407704
54246,1,B,C,H,N,A,A,Y,N,F,...,0,0,0,0,0.683326,1.893137,-0.718982,0.768258,0.679558,-0.921405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14742,1,B,A,H,N,A,A,Y,N,F,...,0,0,1,0,-95.187234,-1.443324,1.187324,-0.015606,-0.891343,2.063250
110514,0,B,A,B,N,A,A,Y,N,F,...,0,0,0,0,-1.280368,-4.493934,-0.436162,-1.439507,-1.613136,-1.945822
205912,1,D,B,E,Z,C,A,Z,Z,F,...,0,0,1,0,-1.365743,-1.591054,-0.875122,-1.265167,-0.085532,-1.520726
88967,1,D,B,E,Z,C,A,Z,Z,F,...,1,0,0,0,-0.986678,-0.223377,-3.411868,-2.682268,-0.609224,-0.390056


In [61]:
#df_train=df_combined.dropna()

In [ ]:
vars_ind.remove()

In [62]:
df_train

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f10_8,f10_9,f10_10,f10_11,x0,x1,x2,x3,x4,x5
0,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,1,0.192403,-1.938869,-1.074925,0.216317,-3.662846,-1.238058
1,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,1,0,0.192403,-0.491062,-0.875122,0.216317,-0.277127,-1.520726
2,0,D,C,H,Z,C,A,Z,Z,F,...,0,0,1,1,0.192403,-3.378403,-1.392959,0.216317,-0.380800,-0.955391
3,1,B,H,H,N,A,A,Y,N,F,...,0,1,0,0,-0.494890,-31.171417,-2.785252,-0.556401,-0.018121,-0.024136
4,0,B,H,H,N,A,A,Y,N,F,...,0,1,0,1,-0.494890,0.615723,-0.313070,-0.556401,0.221019,-0.214726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1,B,A,H,N,A,A,Y,N,F,...,0,1,1,0,1.239706,0.942550,-0.875122,1.393792,0.144381,-1.520726
249996,0,B,A,B,N,A,A,N,N,F,...,0,1,0,1,-0.986678,0.240482,-1.392959,-2.682268,0.565889,-0.955391
249997,1,B,A,B,N,A,A,N,N,F,...,0,1,0,1,-0.986678,0.278152,-1.392959,-2.682268,0.489251,-0.955391
249998,0,B,A,B,N,A,A,N,N,F,...,0,1,0,1,-0.986678,-0.777890,-1.392959,-2.682268,-0.174943,-0.955391


In [64]:
# add folds to randomly distinguish between train and validation sets
rng = np.random.RandomState(2018)
fold = rng.randint(0, 10, df_all.shape[0]) # takes values 0-9 inclusive
df_all['fold'] = fold

In [65]:
idx_train  = df_all['fold'].isin(range(9))
idx_val    = df_all['fold'].isin([9, 10])
idx_design = df_all['fold'].isin(range(10))

print("number of train examples",    np.sum(idx_train == 1))
print("number of val examples",      np.sum(idx_val == 1))
print("number of design examples",   np.sum(idx_design == 1))

number of train examples 9046
number of val examples 954
number of design examples 10000


In [67]:
#initialise h2o connection
h2o.init(ip='localhost', 
         port=54321, 
         https=False,enable_assertions = False)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 6 hours 33 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,2 months and 17 days
H2O_cluster_name:,H2O_from_python_jovyan_w8onbs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.069 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [68]:
#pass df_all to h2o

h2o_df_all = h2o.H2OFrame(df_all[vars_ind + var_dep + ['fold']],
                          destination_frame = 'df_all')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [69]:
#pass index to h2o

idx_h2o_train  = h2o.H2OFrame(idx_train.astype('int').values,  
                              destination_frame = 'idx_h2o_train')
idx_h2o_val    = h2o.H2OFrame(idx_val.astype('int').values  ,  
                              destination_frame = 'idx_h2o_val')
idx_h2o_design = h2o.H2OFrame(idx_design.astype('int').values, 
                              destination_frame = 'idx_h2o_design')


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [70]:
#set manually

h2o_df_design = h2o_df_all[idx_h2o_design, :]
h2o_df_train  = h2o_df_all[idx_h2o_train, :]
h2o_df_val    = h2o_df_all[idx_h2o_val, :]

In [75]:
#pass test df to h2o
h2o_df_test=h2o.H2OFrame(df_test[vars_ind],
                          destination_frame = 'df_test')

KeyError: "['d02_2', 'd02_3'] not in index"

In [161]:
# Finding best model as in part 3 for glm
#Set up the hyper parameters to search over
# We will (stupidly) search over 100 values for alpha - just so we can demonstrate random 
# search with early stopping

alpha_opts = [0.1,0.2,0.4,0.6,0.75,0.99]

#alpha_opts = np.arange(0, 1, 0.).tolist()

hyper_parameters = {"alpha":alpha_opts}

In [162]:
criteria = {# The default strategy, "Cartesian", covers the entire space of h-p combinations. 
            "strategy": "RandomDiscrete", 
            "max_models": 6,
            "stopping_metric": "AUTO",
            "seed": 2021}

grid = H2OGridSearch(H2OGeneralizedLinearEstimator(family="binomial",
                                                   nfolds = 10,
                                                   lambda_search=True),
                     hyper_params=hyper_parameters,
                     grid_id='g5',
                     search_criteria=criteria
                    )

grid.train(y = var_dep,
           x = vars_ind, 
           training_frame=h2o_df_all[idx_h2o_train, :],
           validation_frame=h2o_df_all[idx_h2o_val,:] 
           )

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [164]:
gridglm = grid.get_grid(sort_by='AUC', decreasing=True)
gridglm

      alpha   model_ids                 auc
0     [0.4]  g5_model_3  0.7271663385975469
1    [0.99]  g5_model_2  0.7271206364410034
2     [0.6]  g5_model_4  0.7268291929144829
3    [0.75]  g5_model_6  0.7266378990963563
4     [0.2]  g5_model_5  0.7262986067356209
5     [0.1]  g5_model_1  0.7259440096866927


In [165]:
bstglm = gridglm.models[0]

In [166]:
bstglm

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  g5_model_3


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.4, lambda = 1.843E-4 )","nlambda = 100, lambda.max = 0.1362, lambda.min = 1.843E-4, lambda....",379,241,102,py_6_sid_992c




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.20807910310352987
RMSE: 0.45615688431013496
LogLoss: 0.6019159275553889
Null degrees of freedom: 45049
Residual degrees of freedom: 44808
Null deviance: 62445.44966038139
Residual deviance: 54232.625072740535
AIC: 54716.625072740535
AUC: 0.733830097215758
AUCPR: 0.730814062908424
Gini: 0.46766019443151596

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3496816167448322: 


,,0,1,Error,Rate
0,0,8556.0,14252.0,0.6249,(14252.0/22808.0)
1,1,2687.0,19555.0,0.1208,(2687.0/22242.0)
2,Total,11243.0,33807.0,0.376,(16939.0/45050.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se,...,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2021-07-15 23:06:16,0.000 sec,1,.14E0,1,1.386137,1.386005,1.386178,0.000122,...,,,,,,,,,,
1,,2021-07-15 23:06:16,0.060 sec,2,.12E0,6,1.383758,1.383851,1.386121,0.000114,...,,,,,,,,,,
2,,2021-07-15 23:06:16,0.137 sec,4,.11E0,11,1.378387,1.378803,1.384334,0.000142,...,,,,,,,,,,
3,,2021-07-15 23:06:16,0.203 sec,6,.1E0,13,1.371945,1.372453,1.379404,0.000122,...,,,,,,,,,,
4,,2021-07-15 23:06:16,0.280 sec,8,.94E-1,14,1.365669,1.366212,1.373115,0.000187,...,,,,,,,,,,
5,,2021-07-15 23:06:16,0.360 sec,10,.86E-1,15,1.360217,1.360792,1.366783,0.000339,...,,,,,,,,,,
6,,2021-07-15 23:06:16,0.446 sec,12,.78E-1,16,1.354669,1.355206,1.361233,0.000512,...,,,,,,,,,,
7,,2021-07-15 23:06:17,0.548 sec,14,.71E-1,17,1.349582,1.350161,1.355775,0.000677,...,,,,,,,,,,
8,,2021-07-15 23:06:17,0.635 sec,16,.65E-1,20,1.344411,1.345339,1.350630,0.000839,...,,,,,,,,,,
9,,2021-07-15 23:06:17,0.750 sec,18,.59E-1,20,1.339585,1.340892,1.345524,0.000982,...,,,,,,,,,,



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f13_0,2.597321,1.000000,0.078211
1,f09.F,1.741644,0.670554,0.052445
2,f13,1.655658,0.637448,0.049856
3,f13_1,1.360069,0.523643,0.040955
4,f11,1.191216,0.458633,0.035870
5,f11_0,1.057289,0.407069,0.031837
6,f13_4,1.008754,0.388383,0.030376
7,f03.F,0.853952,0.328782,0.025714
8,f13_3,0.747041,0.287620,0.022495
9,f13_5,0.687308,0.264622,0.020696



See the whole table with table.as_data_frame()


In [187]:
# new model using best parameters

In [83]:
vars_ind_numeric_tree = vars_ind_numeric.copy()
vars_ind_numeric_glm  = vars_ind_numeric.copy()

vars_ind_glm  = vars_ind_categorical + vars_ind_numeric_glm 
vars_ind_tree = vars_ind_categorical + vars_ind_numeric_tree

# Not needed since vars_ind_glm contains the original numeric feature.  But do it anyway...
vars_ind = list(set(vars_ind_glm + vars_ind_tree))

In [78]:
nfolds=10

In [79]:
glm_bst =  H2OGeneralizedLinearEstimator(
                        model_id='glm_bst', 
                        family='binomial',
                        alpha=0.4,
                        lambda_=1.843E-4,
                        standardize=False,
                        early_stopping=True,
                        nfolds=nfolds,
                        fold_assignment="Modulo",               
                        keep_cross_validation_predictions = True, 
                        seed = 2021)

glm_bst.train(x=vars_ind_glm, 
              y='target',
              training_frame=h2o_df_train,
              validation_frame=h2o_df_val)

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [f10_0, c02, e19_0, e17_0, f06_2, e18_0, e16]
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%


In [80]:
glm_bst

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_bst


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.4, lambda = 1.843E-4 )",376,209,3,py_1_sid_b7fc




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.2084895160557699
RMSE: 0.45660652213450686
LogLoss: 0.6028620743415782
Null degrees of freedom: 45049
Residual degrees of freedom: 44840
Null deviance: 62440.53610496649
Residual deviance: 54317.872898176196
AIC: 54737.872898176196
AUC: 0.7328053982456332
AUCPR: 0.7284464103893861
Gini: 0.46561079649126635

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.36428338160994933: 


,,0,1,Error,Rate
0,0,9275.0,13618.0,0.5949,(13618.0/22893.0)
1,1,3061.0,19096.0,0.1382,(3061.0/22157.0)
2,Total,12336.0,32714.0,0.3702,(16679.0/45050.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2021-07-16 15:27:11,0.000 sec,0,31220.268052,0.693014,,,,,,,,,,,,,,
1,,2021-07-16 15:27:12,0.837 sec,1,27307.976530,0.610209,,,,,,,,,,,,,,
2,,2021-07-16 15:27:13,1.292 sec,2,27162.615874,0.607864,,,,,,,,,,,,,,
3,,2021-07-16 15:27:13,1.717 sec,3,27158.936449,0.607829,0.456607,0.602862,0.165819,0.732805,0.728446,1.888954,0.370233,0.455634,0.601464,0.168631,0.735347,0.723043,1.904642,0.363232



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f13_0,2.332463,1.000000,0.077579
1,f09.F,1.623853,0.696197,0.054010
2,f13,1.435874,0.615604,0.047758
3,f13_1,1.274817,0.546554,0.042401
4,f11,1.026648,0.440156,0.034147
5,f11_0,0.875998,0.375568,0.029136
6,f13_4,0.716998,0.307400,0.023848
7,f03.F,0.708077,0.303575,0.023551
8,f11_1,0.672864,0.288478,0.022380
9,f11_2,0.644832,0.276460,0.021447



See the whole table with table.as_data_frame()


In [94]:
y = df_all[var_dep].values.ravel()

In [201]:
model_pred_train = model.predict(h2o_df_all[idx_h2o_train, :]).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%


In [163]:
model_pred_test=model.predict(h2o_df_test).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: ["30146", "B2E8B", "BE271"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: ["Q", "S"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: ["J", "M", "O", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: ["J", "O", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'c09' has levels not trained on: ["D"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'a18' has levels not trained on: ["D"]
  warnings.warn(w)


In [105]:
bst = grid.models[0]

df_bst_preds_train = bst.predict(h2o_df_train).as_data_frame()
df_bst_preds_test = bst.predict(h2o_df_test).as_data_frame()
df_bst_preds_train.head(5)

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: ["30146", "B2E8B", "BE271"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: ["Q", "S"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: ["J", "M", "O", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: ["J", "O", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'c09' has levels not trained on: ["D"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'a18' has levels not trained on: ["D"]
  warnings.warn(w)


,predict,p0,p1
0,1,0.163323,0.836677
1,1,0.094186,0.905814
2,1,0.529998,0.470002
3,1,0.455997,0.544003
4,1,0.515627,0.484373


In [89]:
h2o_df_all[var_dep]= h2o_df_all[var_dep].asfactor()
h2o_df_design[var_dep]= h2o_df_design[var_dep].asfactor()
h2o_df_train[var_dep]= h2o_df_train[var_dep].asfactor()
h2o_df_val[var_dep]= h2o_df_val[var_dep].asfactor()

In [ ]:
# random forest

In [57]:
import h2o

from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [178]:
h2o_df_all[var_dep]= h2o_df_all[var_dep].asfactor()
h2o_df_design[var_dep]= h2o_df_design[var_dep].asfactor()
h2o_df_train[var_dep]= h2o_df_train[var_dep].asfactor()
h2o_df_val[var_dep]= h2o_df_val[var_dep].asfactor()

In [ ]:
#rf grid search

In [81]:
rf_params1 = {  'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate_per_tree': [0.2, 0.5, 1.0],
                'min_rows': [2,5,10,20,40]
             }

search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 10, 'seed': 1}

# Train and validate a random search grid for rf
rf_grid1 = H2OGridSearch(model=H2ORandomForestEstimator(
                                                        seed=2021,
                                                        nfolds= 10,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                          grid_id='rf_grid3',
                          hyper_params=rf_params1,
                          search_criteria=search_criteria)

In [ ]:
rf_grid1.train(x=vars_ind_tree, y='target',
                training_frame=h2o_df_train,
                validation_frame=h2o_df_val,
                ntrees=100,
                seed=1)

In [167]:
rf_grid1sorted = rf_grid1.get_grid(sort_by='auc',decreasing=True)

In [168]:
rf_grid1sorted

    col_sample_rate_per_tree max_depth min_rows sample_rate  \
0                        0.5         9     10.0         0.8   
1                        1.0         9     40.0         0.8   
2                        0.2         9     40.0         0.8   
3                        0.2         5      5.0         0.8   
4                        0.5         5     10.0         0.8   
5                        1.0         5     40.0         0.8   
6                        0.2         3      2.0         0.8   
7                        0.5         3     20.0         0.8   
8                        0.5         3     10.0         0.8   
9                        1.0         3      5.0         1.0   

           model_ids                 auc  
0   rf_grid3_model_1  0.7565483874918617  
1   rf_grid3_model_7  0.7538414859863362  
2   rf_grid3_model_9  0.7507074986939584  
3   rf_grid3_model_5  0.7144700958383214  
4   rf_grid3_model_8  0.7100584405226489  
5   rf_grid3_model_4  0.7040323198398752  
6   r

In [184]:
rfbst = rf_grid1.models[0]

In [ ]:
# rf model using best parameters

In [90]:
rf_bst = H2ORandomForestEstimator(
                        model_id='rf_bst', 
                        ntrees=200,
                        min_rows=10,
                        # Row sample rate per tree: (default: ``0.6320000291``)
                        mtries=-1, # so as not to limit col sample rates
                        max_depth=9,
                        sample_rate=0.8,
                        nfolds=10,
                        fold_assignment = "Modulo",               
                        keep_cross_validation_predictions = True,
                        seed = 2021)

rf_bst.train(x=vars_ind_tree, 
             y='target',
             training_frame=h2o_df_train,
             validation_frame=h2o_df_val)

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [f10_0, c02, e19_0, e17_0, f06_2, e18_0, e16]
  warnings.warn(mesg["message"], RuntimeWarning)


drf Model Build progress: |███████████████████████████████████████████████| 100%


In [91]:
rf_bst

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_bst


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,200.0,200.0,623652.0,9.0,9.0,9.0,144.0,377.0,243.39




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.20996611177573865
RMSE: 0.45822059291976247
LogLoss: 0.6073920945021851
Mean Per-Class Error: 0.3073103909995494
AUC: 0.7591854593559708
AUCPR: 0.7452265230123456
Gini: 0.5183709187119416

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.42818772413153194: 


,,0,1,Error,Rate
0,0,11058.0,11835.0,0.517,(11835.0/22893.0)
1,1,3154.0,19003.0,0.1423,(3154.0/22157.0)
2,Total,14212.0,30838.0,0.3327,(14989.0/45050.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2021-07-16 16:51:00,5 min 9.625 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2021-07-16 16:51:00,5 min 9.776 sec,1.0,0.475613,0.687155,0.666865,0.673630,1.812072,0.466841,0.470724,0.683167,0.680304,0.681049,1.763558,0.457172
2,,2021-07-16 16:51:00,5 min 9.935 sec,2.0,0.471334,0.711395,0.689107,0.684576,1.798949,0.428476,0.461448,0.619630,0.718981,0.706432,1.814675,0.405051
3,,2021-07-16 16:51:01,5 min 10.125 sec,3.0,0.471953,0.693597,0.688793,0.683332,1.768557,0.432141,0.458951,0.614925,0.733529,0.718439,1.904642,0.361616
4,,2021-07-16 16:51:01,5 min 10.313 sec,4.0,0.472725,0.692271,0.685768,0.682443,1.786903,0.412317,0.459514,0.609780,0.737115,0.721643,1.946048,0.359394
5,,2021-07-16 16:51:01,5 min 10.520 sec,5.0,0.471239,0.684546,0.690703,0.688704,1.813289,0.413118,0.457997,0.606824,0.743196,0.729915,1.904642,0.350707
6,,2021-07-16 16:51:01,5 min 10.746 sec,6.0,0.470725,0.677323,0.692566,0.688934,1.774606,0.414154,0.458248,0.607367,0.743542,0.731119,1.904642,0.350707
7,,2021-07-16 16:51:02,5 min 10.960 sec,7.0,0.470315,0.674274,0.694056,0.688475,1.772894,0.411113,0.457599,0.606385,0.750005,0.736036,1.904642,0.359192
8,,2021-07-16 16:51:02,5 min 11.176 sec,8.0,0.469682,0.664583,0.695946,0.689074,1.794809,0.410172,0.457652,0.606691,0.751288,0.737578,1.946048,0.360000
9,,2021-07-16 16:51:02,5 min 11.382 sec,9.0,0.469417,0.662636,0.696901,0.690297,1.785264,0.413085,0.458153,0.607868,0.752652,0.737340,1.904642,0.338384



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f09,23867.191406,1.000000,0.090990
1,e20,13843.264648,0.580012,0.052775
2,f11,11938.401367,0.500201,0.045513
3,f13,11552.454102,0.484031,0.044042
4,f11_0,9859.676758,0.413106,0.037588
5,f11_1,8095.049316,0.339171,0.030861
6,f11_3,7778.015625,0.325887,0.029652
7,f27,6606.561523,0.276805,0.025186
8,f13_0,6538.757324,0.273964,0.024928
9,f11_4,6212.075195,0.260277,0.023682



See the whole table with table.as_data_frame()


In [ ]:
# search grid for xgboost

In [179]:
gbm_params2 = {'learn_rate': [i * 0.01 for i in range(1, 20)],
                'max_depth': list(range(2, 11)),
                'sample_rate': [i * 0.1 for i in range(5, 10)],
                'col_sample_rate': [i * 0.1 for i in range(1, 10)]}

# Search criteria
search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 10, 'seed': 2021}

# Train and validate a random grid of GBMs
gbm_grid5 = H2OGridSearch(model=H2OXGBoostEstimator,
                          grid_id='gbm_grid5',
                          hyper_params=gbm_params2,
                          search_criteria=search_criteria)
gbm_grid5.train(x=vars_ind, y='target',
                training_frame=h2o_df_all[idx_h2o_train, :],
                validation_frame=h2o_df_all[idx_h2o_val,:],
                ntrees=100,
                seed=1)

xgboost Grid Build progress: |████████████████████████████████████████████| 100%


In [204]:
xgb_grid1sorted = gbm_grid5.get_grid(sort_by='mse',decreasing=True)

In [205]:
xgb_grid1sorted 

         col_sample_rate learn_rate max_depth         sample_rate  \
0    0.30000000000000004       0.02         5                 0.8   
1    0.30000000000000004       0.11         2  0.7000000000000001   
2                    0.1       0.18         2                 0.8   
3                    0.9       0.15         3                 0.9   
4                    0.9       0.05         5  0.6000000000000001   
5     0.7000000000000001       0.13        10                 0.5   
6    0.30000000000000004       0.03        10  0.7000000000000001   
7     0.7000000000000001       0.14         7                 0.8   
8                    0.5       0.14         8                 0.8   
9     0.7000000000000001       0.12         9                 0.9   

            model_ids                  mse  
0   gbm_grid5_model_1  0.21204822700154863  
1   gbm_grid5_model_9  0.21002113437638578  
2   gbm_grid5_model_6   0.2079377126822498  
3   gbm_grid5_model_2  0.19841957868054377  
4  gbm_grid5_mo

In [ ]:
# lots of overfitting - big difference between train and validation metrics
# so we try find something which has a decent auc and less overfitting 
# by reducing max_depth and increasing learning_rate, previously (0.99vs0.80)

In [ ]:
#gbm using best parameters

In [92]:
xgb_bst = H2OXGBoostEstimator(model_id='xgb_bst',
                              ntrees=200,
                              learn_rate=0.05,
                              min_rows=1,
                              #col_sample_rate=0.6,
                              col_sample_rate_per_tree=0.7,
                              sample_rate=0.9,
                              max_depth=6,
                              
                              nfolds=nfolds,
                              fold_assignment="Modulo",          
                              keep_cross_validation_predictions=True,
                              seed=2021)

xgb_bst.train(x=vars_ind, 
              y='target',
              training_frame=h2o_df_train,
              validation_frame=h2o_df_val)

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [f10_0, c02, e19_0, e17_0, f06_2, e18_0, e16]
  warnings.warn(mesg["message"], RuntimeWarning)


xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [93]:
xgb_bst

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  xgb_bst


Model Summary: 


,,number_of_trees
0,,200.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.1625747153572898
RMSE: 0.4032055497600322
LogLoss: 0.49630964404570244
Mean Per-Class Error: 0.22752030452728245
AUC: 0.8566228202011142
AUCPR: 0.8494852111387073
Gini: 0.7132456404022285

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4475790128959873: 


,,0,1,Error,Rate
0,0,16093.0,6800.0,0.297,(6800.0/22893.0)
1,1,3687.0,18470.0,0.1664,(3687.0/22157.0)
2,Total,19780.0,25270.0,0.2328,(10487.0/45050.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2021-07-16 17:00:13,8 min 2.330 sec,0.0,0.500000,0.693147,0.500000,0.491831,1.000000,0.508169,0.500000,0.693147,0.500000,0.483030,1.000000,0.516970
1,,2021-07-16 17:00:15,8 min 3.503 sec,1.0,0.496476,0.686121,0.713137,0.706530,1.827599,0.395694,0.496634,0.686436,0.703316,0.684905,1.807693,0.409091
2,,2021-07-16 17:00:15,8 min 3.831 sec,2.0,0.493137,0.679502,0.737762,0.728373,1.840365,0.379001,0.493196,0.679618,0.736480,0.714430,1.816555,0.368889
3,,2021-07-16 17:00:15,8 min 4.167 sec,3.0,0.491015,0.675305,0.747059,0.740167,1.872501,0.336559,0.491078,0.675428,0.742718,0.727241,1.883272,0.358586
4,,2021-07-16 17:00:16,8 min 4.960 sec,4.0,0.488851,0.671029,0.752304,0.745793,1.977513,0.335050,0.488936,0.671195,0.747314,0.733264,1.911012,0.356364
5,,2021-07-16 17:00:18,8 min 6.876 sec,5.0,0.486270,0.665938,0.752081,0.746239,1.980564,0.339001,0.486272,0.665942,0.749435,0.735937,1.907890,0.359798
6,,2021-07-16 17:00:22,8 min 10.973 sec,22.0,0.461231,0.615732,0.773466,0.765610,2.001660,0.320400,0.461276,0.615895,0.771523,0.758173,1.946048,0.340404
7,,2021-07-16 17:00:26,8 min 15.131 sec,39.0,0.448030,0.588377,0.792369,0.784789,2.015185,0.311521,0.449093,0.590697,0.786686,0.772138,1.946048,0.313535
8,,2021-07-16 17:00:30,8 min 19.198 sec,56.0,0.437554,0.566928,0.808213,0.799897,2.015185,0.283663,0.440083,0.572186,0.798897,0.783540,1.946048,0.294545
9,,2021-07-16 17:00:34,8 min 23.359 sec,71.0,0.431334,0.554063,0.815803,0.807754,2.024201,0.279267,0.435246,0.562020,0.803450,0.789291,1.946048,0.288485



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f11_0,15647.166016,1.000000,0.092879
1,f09.F,9904.045898,0.632961,0.058788
2,f13,8768.502930,0.560389,0.052048
3,f27.F,5945.305176,0.379961,0.035290
4,f13_1,5429.235840,0.346979,0.032227
5,e20.A4893,4968.520020,0.317535,0.029492
6,f13_4,4937.550781,0.315556,0.029308
7,f09.C,4449.252930,0.284349,0.026410
8,f11,4388.908691,0.280492,0.026052
9,e02_2,4031.025146,0.257620,0.023927



See the whole table with table.as_data_frame()


In [ ]:
# this reduces overfitting while keeping the mse and auc similar to what it was before on the validation set

In [ ]:
# ensemble using the 3 models above and default metalearner - auto

In [98]:
h2o_df_train.shape

(45050, 197)

In [99]:
h2o_df_val.shape

(4950, 197)

In [96]:
h2o_df_val

f06,a12,c06,e18_3,e18_10,e05_4,e19_13,f02_2,e06,e14,f01_2,e04_1,f13_4,e21,a19,e01,f06_0,c09,f23,f19,f11_0,e17_5,e25,e08_0,f06_2,f16,e02_2,e06_3,e17_2,f21,f27,e22,f02_4,f30,e12_5,f18,f10_9,f13_1,f10_5,e17_4,b07,a03,e18_0,e06_2,f09,e02_5,e09_2,a10,f10_6,c07,f10_11,e06_0,e12,e19_14,e08_4,e12_3,e19_3,e18_4,f02_5,f11_4,f10_4,f13_3,f10_0,e24,e18_2,e18_7,a17,e17_0,e15_1,e19_6,f04,c08,f02,f24,c02,f33,e03,f10_2,f01_3,e08,x1,a16,e05_3,e09_0,x0,e05_0,f02_3,e06_1,e08_3,e08_2,e04,e05,e09_1,f13,b04,f10_1,e19_15,f29,e05_1,e09_3,e02,x2,e17_1,e20,e17_6,a01,e12_1,e09,e02_0,e19_7,a18,e04_4,e15_0,e12_4,f05,e19_11,e02_4,f06_1,e18_9,e18_6,f32,e07,f31,a13,f01_1,e04_3,f25,e19_0,e18_8,e19_2,f34,f10_8,f10_10,f11_5,e19_8,e19_10,a20,c04,e05_2,f13_5,x4,e18_5,e17_3,e15_5,f15,f17,e04_2,f02_1,c05,e19_9,b03,e02_1,e19_5,e11,e18_1,f13_2,e15_4,e05_5,e19_12,f20,f26,e17_7,e16,f10_7,b02,e02_3,f03,f11_3,e15_3,f22,e12_0,f07,e13,e04_5,e12_2,e15,e04_0,f01,f08,f13_0,e19_4,e15_2,x3,f11_2,x5,f28,e08_1,f01_0,e23,f11,f10_3,f02_0,f11_1,e19_1,a02,target,fold
0.516294,A,A,0,1,0,1,0.934864,0.962475,A,0,2.18626,0,L,N,A,2.47779,A,-0.0462924,-0.106451,0,1,A,0,0,-0.250323,1.21193,0,0,-0.0857895,F,A,0,A,0,-0.0844011,0,0.765339,0,0,A,H,0,0,B,0,0,N,0,A,0,2.0228,0.825947,1,0,0.483467,1,0,0,0,1,0,0,A,0,1,N,0,0.489375,0,B,A,0.599132,-0.0796243,Z,N,A,0,0,-1.20813,-0.0229843,Y,0,0,1.12209,0,0.311621,0.541822,0,0,1.07999,-1.49034,0,-0.0137706,G,0,0,D,0,0,0.817002,0.546175,0,A4893,1,B,1.83027,-1.15361,2.21609,1,F,0,0.803974,0,A,0,0,1.99195,0,0,0.098285,-0.131137,0.110374,A,0,0.728755,-0.0810681,0,0,0,N,0,0,0,0,0,N,A,0,0,-0.0082504,0,0,0,-0.331464,-0.198479,1.45751,1.66198,A,0,N,1.8352,1,D,0,0,0,0,1,-0.247492,-0.0785237,1,nan,0,N,0.450144,C,0,0,-0.208867,2.21014,A,E,0,1.20867,-0.468323,2.39448,-1.30165,0.193508,1.53068,0,0,1.03946,0,2.43662,0.181275,0,0,-0.50674,-2.11216,0,1.9736,0,0,C,0,9
0.516294,A,A,0,1,0.59,0,1.86973,-1.06033,A,0,0.555242,0,N,N,A,2.47779,A,-0.0462924,-0.106451,2.20525,0,A,0.850524,0,-0.250323,2.04296,0,0,-0.0857895,F,A,0.519369,A,0,-0.0844011,1,1.14801,0,1,A,H,0,0,G,0.277012,0,N,0,A,1,0,-0.520855,0,0,0,0,0,0.138498,0,0,0,0,A,0,1,N,0,1.57299,1,B,B,1.534,-0.0796243,Z,N,A,0,0,-0.357607,0.240482,Y,1.28412,0.894335,0.519685,2.9847,1.24648,0,0,0,-0.551036,1.59848,0,0.368899,A,0,0,B,2.63765,0,1.64804,-0.31307,0,A5DA8,1,B,0.483467,-0.259277,3.04713,0,F,0,1.88759,0,A,0,0.58865,1.99195,0,0,0.098285,-0.131137,0.110374,A,0.355377,0,-0.0810681,0,0,0,N,0,0,0,0,0,N,A,1.94353,0,0.565889,1,0,0,-0.331464,-0.198479,0,2.59684,A,1,N,2.66624,1,E,0,0.38267,0,0.208235,1,-0.247492,-0.0785237,1,nan,0,N,1.28118,C,0.490056,0.279643,-0.208867,0.863334,A,E,0,0,0.615293,0.763458,0.0487816,0.193508,1.91335,1,1.08362,0.584278,0.980113,-0.214726,0.181275,0,1.35043,0.552313,0.828174,0,2.90846,1.7152,0,A,1,9
0.516294,A,A,0,0,0.242941,1,1.17724,-1.06033,D,0.568603,0,1.14801,M,N,A,2.47779,B,-0.0462924,-0.106451,0.735085,1,A,0.850524,0,-0.250323,0,0,0,-0.0857895,F,A,0,A,0,-0.0844011,0,2.67869,0,0,A,B,0,0,B,0,0,N,1,A,0,0,-0.866188,1,0,0,0,0,0,0,1,1.53068,0,A,0,1,N,0,0.489375,0,B,A,0.841504,-0.0796243,Z,N,A,1,0,-0.357607,2.25736,N,0.937058,0.894335,1.53426,2.63765,0.553993,0,0,0,-1.62682,1.25142,0,1.89958,F,0,0,B,2.29059,0,-1.50297,0.403861,0,A4893,1,B,0.138134,-0.259277,0,0,F,0,0.803974,0,A,0,0,1.99195,1,1,0.098285,-0.131137,0.110374,A,1.20828,0,-0.0810681,0,1,0,N,1,1,0,0,1,N,A,1.59647,0.765339,1.5985,0,0,0,-0.331464,-0.198479,0,1.90435,A,0,N,0,0,B,0,1.91335,0,0,0,-0.247492,-0.0785237,0,nan,1,N,0,C,0,0,-0.208867,0.518001,A,E,0,0,-0.468323,0,0.901686,0.193508,3.44403,0,0,1.72496,0,0.166454,0.181275,0,2.20334,-0.50674,-0.641995,0,2.21597,0.245028,0,A,0,9
0.516294,A,A,0,0,0,1,0.623242,-1.06033,B,0,1.63102,0,C,N,A,2.47779,A,-0.0462924,-0.106451,0.735085,1,A,0,0,-0.250323,0,0,0,-0.0857895,F,A,0,A,0,-0.0844011,0,1.53068,0,0,A,H,0,0,B,0,0,N,0,A,1,0,-1.07339,1,0,0,0,0,0,0,1,0.38267,0,A,0,1,N,0,0.489375,1,B,B,0.28751,-0.0796243,Z,N,A,0,0,-1.20813,2.61406,Y,0.0694117,0,-0.49489,1.77,0,0,0,0,0.524745,0.383773,0,0.751569,C

In [102]:
h2o_df_train

f06,a12,c06,e18_3,e18_10,e05_4,e19_13,f02_2,e06,e14,f01_2,e04_1,f13_4,e21,a19,e01,f06_0,c09,f23,f19,f11_0,e17_5,e25,e08_0,f06_2,f16,e02_2,e06_3,e17_2,f21,f27,e22,f02_4,f30,e12_5,f18,f10_9,f13_1,f10_5,e17_4,b07,a03,e18_0,e06_2,f09,e02_5,e09_2,a10,f10_6,c07,f10_11,e06_0,e12,e19_14,e08_4,e12_3,e19_3,e18_4,f02_5,f11_4,f10_4,f13_3,f10_0,e24,e18_2,e18_7,a17,e17_0,e15_1,e19_6,f04,c08,f02,f24,c02,f33,e03,f10_2,f01_3,e08,x1,a16,e05_3,e09_0,x0,e05_0,f02_3,e06_1,e08_3,e08_2,e04,e05,e09_1,f13,b04,f10_1,e19_15,f29,e05_1,e09_3,e02,x2,e17_1,e20,e17_6,a01,e12_1,e09,e02_0,e19_7,a18,e04_4,e15_0,e12_4,f05,e19_11,e02_4,f06_1,e18_9,e18_6,f32,e07,f31,a13,f01_1,e04_3,f25,e19_0,e18_8,e19_2,f34,f10_8,f10_10,f11_5,e19_8,e19_10,a20,c04,e05_2,f13_5,x4,e18_5,e17_3,e15_5,f15,f17,e04_2,f02_1,c05,e19_9,b03,e02_1,e19_5,e11,e18_1,f13_2,e15_4,e05_5,e19_12,f20,f26,e17_7,e16,f10_7,b02,e02_3,f03,f11_3,e15_3,f22,e12_0,f07,e13,e04_5,e12_2,e15,e04_0,f01,f08,f13_0,e19_4,e15_2,x3,f11_2,x5,f28,e08_1,f01_0,e23,f11,f10_3,f02_0,f11_1,e19_1,a02,target,fold
-1.57282,A,A,0,0,0,0,0,-1.06033,B,0,1.24929,1.14801,F,N,A,0.388673,A,-0.0462924,-0.106451,1.22514,0,A,1.80736,0,-0.250323,0,0,0,-0.0857895,A,A,0,A,0,-0.0844011,1,2.67869,1,1,A,B,0,0,B,0,0.21464,N,0,A,0,0,-0.279121,0,0,0,0,0,0,0,0,1.53068,0,A,0,1,N,0,0.174777,1,A,A,-1.2706,-0.0796243,Z,N,A,0,0,0.599232,-1.49503,N,0,1.78867,-0.13488,1.18,0,0,0,0.212631,0.143016,-0.206226,0.894335,1.89958,B,0,0,E,0.832941,0,-1.57223,-4.17969,0,A4893,0,B,0.725201,0.635058,0,0,F,0,0.489375,0,A,1,0,0,0,1,0.098285,-0.131137,0.110374,A,0.46199,0,-0.0810681,0,0,0,N,0,1,0,1,1,N,A,0.138823,0.765339,-2.41359,0,0,0,-0.331464,-0.198479,0.520539,0,A,0,N,0,0,F,0,1.91335,0,0,0,-0.247492,-0.0785237,0,nan,1,N,0,A,0,0,-0.208867,1.10507,A,E,0,0.1036,-0.782922,1.45751,0.155395,0.193508,3.44403,0,0,-0.151644,0,-0.0964901,0.181275,0.95684,1.45705,1.61137,-0.151939,0,0.103874,0.735085,0,A,0,6
0.516294,A,A,1,0,0,0,0.519369,1.5043,A,0,2.7068,1.91335,B,N,A,2.47779,A,-0.0462924,-0.106451,3.43039,0,A,2.4807,0,-0.250323,1.48894,0.0722429,0,-0.0857895,F,A,0,A,0,-0.0844011,0,3.44403,1,1,A,H,0,0.541822,C,0,0,N,1,A,0,2.56462,0.13528,0,0,0,0,1,0,1.22514,1,2.29602,0,D,0,0,N,0,0.489375,1,A,A,0.183637,-0.0796243,Z,N,D,0,0,1.27256,14.5119,Y,0,0,1.06397,0.138823,0,1.08364,0.177193,0.885963,1.60053,-1.2474,0,2.66492,B,0,0,D,0,0,1.09401,-0.561829,0,D53C5,0,B,1.1396,-1.15361,2.49311,0,F,0.520539,0.803974,0,A,1,0.0346265,1.99195,1,1,0.098285,-0.131137,0.110374,A,0,1.24929,-0.0810681,0,1,0,N,0,1,0.735085,0,1,N,A,0,1.53068,0.489376,1,0,0,-0.331464,-0.198479,1.97805,1.24648,A,1,N,2.11222,0,F,0,2.67869,0,0,1,-0.247492,-0.0785237,1,nan,1,N,0.727156,A,1.7152,0,-0.208867,1.51947,A,E,0.242918,0.518001,-0.468323,2.91502,-0.590897,0.193508,4.20937,1,0,2.40767,2.20525,-2.36873,0.181275,1.63017,0.710754,1.61137,2.05331,0,1.55811,2.94034,0,C,1,2
0.516294,A,A,0,1,0,0,0,-1.06033,C,0,0.728755,0,H,N,A,2.47779,A,-0.0462924,-0.106451,3.18537,1,A,0.850524,0,-0.250323,1.21193,0,0,-0.0857895,F,A,0,A,0,-0.0844011,1,1.14801,0,0,A,H,0,0,F,0,0,N,1,A,0,0,0.618747,0,0,0.276267,0,0,0,0.980113,0,0,0,A,0,0,N,0,2.48183,0,B,A,-0.335732,-0.0796243,Z,N,A,0,0,-0.357607,-1.09879,N,0,0.894335,0.356044,1.70059,0,0,0,0,-0.377523,0.314362,0,0.368899,B,0,0,B,1.35353,0,0.817002,-0.143383,0,C8FEF,1,B,1.62307,-0.259277,2.21609,0,F,0,2.79643,0,A,1,0,1.99195,0,1,0.098285,-0.131137,0.110374,A,0,0,-0.0810681,0,1,0,N,1,1,0.490056,1,0,N,A,0.659411,0,-0.123851,0,0,0.139822,-0.331464,-0.198479,0,0.727116,A,1,N,1.8352,0,B,0,0.38267,0.349554,0,1,-0.247492,-0.0785237,1,nan,0,N,0.450144,C,1.47017,1.18848,-0.208867,2.00294,A,E,0,1.00147,1.52413,0.936971,-0.590897,0.193508,1.91335,0,1.99246,0.400297,1.96023,-0.468847,0.181275,0,0.710754,-0.50674,1.80829,1,1.03874,2.69531,0,A,1,5
0.516294,A,A,1,1,0,0,1.59273,-1.06033,A,1.38597,2.81091,0.765339,O,N,A,2.47779,A,-0.0462924,-0.106451,1.7152,1,A,2.55157,0,2.95821,2.07759,0,0,-0.0857895,F,A,0.242372,A,0,-0.0844011,0,2.29602,1,1,A,B,0,0,C,0.311638,0.930108,N,1,A,0,0,-1.31512,0,0,0,1,0,0,0,1,1.14801

In [100]:
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[glm_bst, rf_bst,xgb_bst])
ensemble.train(x=vars_ind, y='target',
                training_frame=h2o_df_train,
                validation_frame=h2o_df_val)

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping unused columns: [f10_0, c02, e19_0, e17_0, f06_2, e18_0, e16]
  warnings.warn(mesg["message"], RuntimeWarning)


stackedensemble Model Build progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_9ef658d356605c99881e30f4a7a744fe failed with an exception: java.lang.IllegalArgumentException: Test/Validation dataset has categorical column 'target' which is real-valued in the training data
stacktrace: 
java.lang.IllegalArgumentException: Test/Validation dataset has categorical column 'target' which is real-valued in the training data
	at hex.Model.adaptTestForTrain(Model.java:1560)
	at hex.Model.adaptTestForTrain(Model.java:1404)
	at hex.Model.adaptTestForTrain(Model.java:1400)
	at hex.Model.score(Model.java:1697)
	at hex.Model.score(Model.java:1689)
	at hex.ensemble.StackedEnsemble$StackedEnsembleDriver.buildPredictionsForBaseModel(StackedEnsemble.java:348)
	at hex.ensemble.StackedEnsemble$StackedEnsembleCVStackingDriver.getPredictionsForBaseModel(StackedEnsemble.java:463)
	at hex.ensemble.StackedEnsemble$StackedEnsembleDriver.prepareLevelOneFrame(StackedEnsemble.java:327)
	at hex.ensemble.StackedEnsemble$StackedEnsembleDriver.compu

In [213]:
ensemble #less overfitting in xgboost leads to less overfitting in ensemble 

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  my_ensemble_binomial

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.1527329118161351
RMSE: 0.39081058304009003
LogLoss: 0.4705474975394103
Null degrees of freedom: 10029
Residual degrees of freedom: 10028
Null deviance: 13902.999806891223
Residual deviance: 9439.18280064057
AIC: 9443.18280064057
AUC: 0.8642884437327184
AUCPR: 0.8597407199768403
Gini: 0.7285768874654368

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38868423595571727: 


,,0,1,Error,Rate
0,0,3432.0,1645.0,0.324,(1645.0/5077.0)
1,1,646.0,4307.0,0.1304,(646.0/4953.0)
2,Total,4078.0,5952.0,0.2284,(2291.0/10030.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.388684,0.789913,247.0
1,max f2,0.227473,0.865733,319.0
2,max f0point5,0.581564,0.792955,163.0
3,max accuracy,0.509203,0.783848,194.0
4,max precision,0.948080,1.000000,0.0
5,max recall,0.058387,1.000000,397.0
6,max specificity,0.948080,1.000000,0.0
7,max absolute_mcc,0.509203,0.568128,194.0
8,max min_per_class_accuracy,0.481425,0.778720,206.0
9,max mean_per_class_accuracy,0.509203,0.783492,194.0



Gains/Lift Table: Avg response rate: 49.38 %, avg score: 48.98 %


...

,,0,1,Error,Rate
0,0,1396.0,1123.0,0.4458,(1123.0/2519.0)
1,1,340.0,2091.0,0.1399,(340.0/2431.0)
2,Total,1736.0,3214.0,0.2956,(1463.0/4950.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.348085,0.740833,267.0
1,max f2,0.154686,0.844940,354.0
2,max f0point5,0.611556,0.734968,145.0
3,max accuracy,0.469390,0.728889,212.0
4,max precision,0.946334,1.000000,0.0
5,max recall,0.053749,1.000000,398.0
6,max specificity,0.946334,1.000000,0.0
7,max absolute_mcc,0.469390,0.458691,212.0
8,max min_per_class_accuracy,0.487267,0.725216,204.0
9,max mean_per_class_accuracy,0.469390,0.729256,212.0



Gains/Lift Table: Avg response rate: 49.11 %, avg score: 49.01 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010101,0.938050,1.995475,1.995475,0.980000,0.940925,0.980000,0.940925,0.020156,0.020156,99.547511,99.547511,0.019759
1,2,0.020000,0.933923,2.036199,2.015631,1.000000,0.935846,0.989899,0.938411,0.020156,0.040313,103.619910,101.563143,0.039916
2,3,0.030101,0.930159,1.954751,1.995202,0.960000,0.932295,0.979866,0.936359,0.019745,0.060058,95.475113,99.520180,0.058867
3,4,0.040000,0.922669,1.828424,1.953928,0.897959,0.926621,0.959596,0.933949,0.018100,0.078157,82.842368,95.392842,0.074981
4,5,0.050101,0.911808,1.751131,1.913042,0.860000,0.917518,0.939516,0.930636,0.017688,0.095845,75.113122,91.304189,0.089891
5,6,0.100000,0.864441,1.838350,1.875771,0.902834,0.888395,0.921212,0.909558,0.091732,0.187577,83.834979,87.577129,0.172095
6,7,0.150101,0.816443,1.748832,1.833401,0.858871,0.841138,0.900404,0.886721,0.087618,0.275195,74.883229,83.340134,0.245819
7,8,0.200000,0.770114,1.475626,1.744138,0.724696,0.794302,0.856566,0.863663,0.073632,0.348828,47.562606,74.413821,0.292456
8,9,0.300000,0.669295,1.472645,1.653640,0.723232,0.723059,0.812121,0.816795,0.147265,0.496092,47.264500,65.364048,0.385334
9,10,0.400000,0.564795,1.271082,1.558001,0.624242,0.615573,0.765152,0.766489,0.127108,0.623200,27.108186,55.800082,0.438603


In [ ]:
# ensemble using different metalearner - gbm

In [249]:
level_0 = [glm_bst, rf_bst, xgb_bst]

h2o.remove('ens_1')

ens_1 = H2OStackedEnsembleEstimator(model_id="ens_1",
                                    base_models=level_0,
                                    metalearner_algorithm="gbm",
                                    metalearner_nfolds=nfolds,
                                    metalearner_fold_assignment='Modulo',
                                    seed=2020)

ens_1.train(x=vars_ind, 
            y='target',
            training_frame=h2o_df_train,
                validation_frame=h2o_df_val)

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping unused columns: [f10_0, c02, e19_0, e17_0, e16, e18_0, f06_2, a20]
  warnings.warn(mesg["message"], RuntimeWarning)


stackedensemble Model Build progress: |███████████████████████████████████| 100%


In [216]:
ens_1

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ens_1

No model summary for this model

ModelMetricsBinomial: stackedensemble
** Reported on train data. **

MSE: 0.1505832218050687
RMSE: 0.3880505402715846
LogLoss: 0.4625648112849653
Mean Per-Class Error: 0.21328942526330508
AUC: 0.868468272315941
AUCPR: 0.8709426110427693
Gini: 0.7369365446318821

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4056844978386691: 


,,0,1,Error,Rate
0,0,3393.0,1592.0,0.3194,(1592.0/4985.0)
1,1,669.0,4297.0,0.1347,(669.0/4966.0)
2,Total,4062.0,5889.0,0.2272,(2261.0/9951.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.405684,0.791709,237.0
1,max f2,0.264970,0.868849,300.0
2,max f0point5,0.610168,0.798993,145.0
3,max accuracy,0.521871,0.786755,186.0
4,max precision,0.970819,1.000000,0.0
5,max recall,0.040866,1.000000,394.0
6,max specificity,0.970819,1.000000,0.0
7,max absolute_mcc,0.521871,0.574074,186.0
8,max min_per_class_accuracy,0.503897,0.785557,195.0
9,max mean_per_class_accuracy,0.521871,0.786711,186.0



Gains/Lift Table: Avg response rate: 49.90 %, avg score: 49.65 %


...

,,0,1,Error,Rate
0,0,12947.0,9861.0,0.4323,(9861.0/22808.0)
1,1,3334.0,18908.0,0.1499,(3334.0/22242.0)
2,Total,16281.0,28769.0,0.2929,(13195.0/45050.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.363755,0.741330,257.0
1,max f2,0.162257,0.845716,347.0
2,max f0point5,0.604223,0.735287,151.0
3,max accuracy,0.487712,0.729145,204.0
4,max precision,0.957037,0.964286,2.0
5,max recall,0.032602,1.000000,398.0
6,max specificity,0.964163,0.999956,0.0
7,max absolute_mcc,0.487712,0.458663,204.0
8,max min_per_class_accuracy,0.499883,0.727408,199.0
9,max mean_per_class_accuracy,0.487712,0.729305,204.0



Gains/Lift Table: Avg response rate: 49.37 %, avg score: 49.38 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010411,0.949310,1.952030,1.952030,0.963753,0.954661,0.963753,0.954661,0.020322,0.020322,95.203028,95.203028,0.019577
1,2,0.020000,0.942710,1.917611,1.935527,0.946759,0.946090,0.955605,0.950551,0.018389,0.038711,91.761103,93.552738,0.036957
2,3,0.030411,0.938367,1.917481,1.929349,0.946695,0.940405,0.952555,0.947078,0.019962,0.058673,91.748108,92.934948,0.055823
3,4,0.040000,0.932899,1.945742,1.933279,0.960648,0.935980,0.954495,0.944417,0.018658,0.077331,94.574225,93.327938,0.073736
4,5,0.050033,0.923936,1.940307,1.934689,0.957965,0.929026,0.955191,0.941331,0.019468,0.096799,94.030687,93.468862,0.092371
5,6,0.100000,0.880441,1.831991,1.883374,0.904487,0.904717,0.929856,0.923036,0.091539,0.188337,83.199059,88.337380,0.174483
6,7,0.150011,0.818718,1.698211,1.821644,0.838438,0.847992,0.899379,0.898018,0.084929,0.273267,69.821130,82.164383,0.243453
7,8,0.200000,0.762854,1.583842,1.762207,0.781972,0.789496,0.870033,0.870893,0.079175,0.352441,58.384227,76.220664,0.301100
8,9,0.300000,0.664700,1.467044,1.663819,0.724306,0.715649,0.821458,0.819145,0.146704,0.499146,46.704433,66.381920,0.393350
9,10,0.400000,0.571262,1.234601,1.556515,0.609545,0.614446,0.768479,0.767970,0.123460,0.622606,23.460120,55.651470,0.439688


In [ ]:
# gbm metalearner does slightly better in terms of auc and MSE of validation set

In [ ]:
level_0 = [glm_bst, rf_bst, xgb_bst]

h2o.remove('ens_1')

ens_1 = H2OStackedEnsembleEstimator(model_id="ens_1",
                                    base_models=level_0,
                                    metalearner_algorithm="gbm",
                                    metalearner_nfolds=nfolds,
                                    metalearner_fold_assignment='Modulo',
                                    seed=2020)

ens_1.train(x=vars_ind, 
            y='target',
            training_frame=h2o_df_train,
                validation_frame=h2o_df_val)

In [251]:
pred = ens_1.predict(df_test[vars_ind])

ValueError: test_data must be an instance of H2OFrame

In [217]:
df_pred_glm = glm_bst.cross_validation_holdout_predictions().as_data_frame()
df_pred_rf  = rf_bst.cross_validation_holdout_predictions().as_data_frame()
df_pred_xgb = xgb_bst.cross_validation_holdout_predictions().as_data_frame()


In [221]:
df_level_0_train = pd.concat([df_pred_glm, df_pred_rf, df_pred_xgb], axis=1)

In [227]:
df_level_0_train

,predict,p0,p1,predict,p0,p1,predict,p0,p1
0,1,0.114966,0.885034,1,0.090696,0.909304,1,0.034866,0.965134
1,1,0.413976,0.586024,1,0.475272,0.524728,1,0.499591,0.500409
2,1,0.330375,0.669625,1,0.375009,0.624991,1,0.169667,0.830333
3,1,0.402055,0.597945,1,0.512646,0.487354,1,0.217924,0.782076
4,1,0.565953,0.434047,1,0.558007,0.441993,0,0.712597,0.287403
...,...,...,...,...,...,...,...,...,...
45045,1,0.456580,0.543420,1,0.393282,0.606718,1,0.390155,0.609845
45046,1,0.422082,0.577918,0,0.566002,0.433998,0,0.565965,0.434035
45047,0,0.692205,0.307795,0,0.726941,0.273059,0,0.783974,0.216026
45048,1,0.352010,0.647990,1,0.389006,0.610994,1,0.170945,0.829055


In [230]:
df_level_0_train=df_level_0_train.drop(['predict', 'p0'], axis=1)

In [236]:
df_level_0_train

,glm,rf,gbm
0,0.885034,0.909304,0.965134
1,0.586024,0.524728,0.500409
2,0.669625,0.624991,0.830333
3,0.597945,0.487354,0.782076
4,0.434047,0.441993,0.287403
...,...,...,...
45045,0.543420,0.606718,0.609845
45046,0.577918,0.433998,0.434035
45047,0.307795,0.273059,0.216026
45048,0.647990,0.610994,0.829055


In [239]:
y_train=h2o_df_train[var_dep]

In [116]:
%%time
dict_ = {'df_train': df_train,
         'df_test': df_test}

f_name = dirPData + '10_df_250k.pickle'
with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
del f_name, dict_

CPU times: user 2.76 s, sys: 3.43 s, total: 6.19 s
Wall time: 6.2 s


In [69]:
# store variable names to pickle
dict_ = {'vars_ind_numeric': vars_ind_numeric,
         'vars_ind_categorical': vars_ind_categorical,
         'vars_ind_hccv': vars_ind_hccv,
         'vars_notToUse': vars_notToUse,
         'var_dep': var_dep,
         'vars_ind': vars_ind}

f_name = dirPData + '02_vars.pickle'
with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
del f_name, dict_

In [164]:
df_sub_14 = pd.DataFrame({
        "unique_id": df_test["unique_id"],
        "Predicted": model_pred_test['p1']
    })

In [165]:
df_sub_14.to_csv('../POutput/df_sub_14.csv', index=False)

In [76]:
df_sub_13

,unique_id,Predicted
0,6,0.575233
1,16,0.926723
2,17,0.366716
3,18,0.698630
4,19,0.606801
...,...,...
296685,2265630,0.518623
296686,2265631,0.363305
296687,2265632,0.633529
296688,2265637,0.245551
